In [3]:
from utils.ModelWrapper import ModelWrapper
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from utils.dot_dict import Dotdict
import wandb

In [4]:
# Init
data_dir = "./data/data-large/data"
img_size = 128

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


### Wandb Sweep training of the model

In [6]:

config_defaults = {
      'epochs': 1,
      'batch_size': 256,
      'learning_rate': 0.0001,
      'optimizer': 'nadam',
      'hidden_layer_size': 64,
      'conv_layer_1_size': 16,
      'conv_layer_2_size': 32,
      'conv_layer_3_size': 64,
      'dropout': 0.5,
   }

# # Config is a variable that holds and saves hyperparameters and inputs
config = Dotdict(config_defaults)

# init wrapper
model_wrapper = ModelWrapper(data_dir, img_size, config.batch_size, use_generator=False)

data_augmentation = Sequential(
      [
         layers.RandomFlip("horizontal", input_shape=(img_size, img_size, 3)),
         layers.RandomRotation(0.1),
         layers.RandomZoom(0.1),
      ]
   )

input_layers = [
      data_augmentation if config.use_augmentation else None,
   layers.Conv2D(filters = config.conv_layer_1_size, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(img_size, img_size, 3)),
   layers.MaxPooling2D(),
   layers.Conv2D(filters = config.conv_layer_2_size, kernel_size=(3, 3), padding='same', activation='relu'),
   layers.MaxPooling2D(),
   layers.Conv2D(filters = config.conv_layer_3_size, kernel_size=(3, 3), padding='same', activation='relu'),
   layers.MaxPooling2D(),
   layers.Dropout(config.dropout) if config.dropout > 0 else None,
   layers.Flatten(),
   layers.Dense(config.hidden_layer_size, activation='relu'),
   layers.Dense(units = 1, activation='sigmoid')
]
input_layers = [e for e in input_layers if e is not None]

# create model
model_wrapper.create_model(None, input_layers, config, use_wandb=False)

# Define the optimizer
if config.optimizer=='sgd':
   optimizer = tf.keras.optimizers.SGD(learning_rate=config.learning_rate) 
elif config.optimizer=='rmsprop':
   optimizer = tf.keras.optimizers.RMSprop(learning_rate=config.learning_rate,)
elif config.optimizer=='adam':
   optimizer = tf.keras.optimizers.Adam(learning_rate=config.learning_rate,) 
elif config.optimizer=='nadam':
   optimizer = tf.keras.optimizers.Nadam(learning_rate=config.learning_rate,) 

# Compile the model
model_wrapper.model.compile(optimizer=optimizer,loss='binary_crossentropy', metrics=['accuracy'])
# Train the model
model_wrapper.fit()

Found 29988 files belonging to 2 classes.
Found 3431 files belonging to 2 classes.
Found 1587 files belonging to 2 classes.
 30/118 [======>.......................] - ETA: 2:48 - loss: 3.7713 - accuracy: 0.5098

KeyboardInterrupt: 

### Evaluate model

In [7]:
# load the bess model NB: edit namd and path
loaded_model = wandb.restore('model-best.h5', run_path="dat550/deepfake-efficientnet/runs/29425ckm")
best_model = load_model(loaded_model.name)

AttributeError: 'ModelWrapper' object has no attribute 'name'

In [8]:
model_wrapper = ModelWrapper(data_dir, img_size, 32)
model_wrapper.evaluate_model(best_model)

Found 29988 images belonging to 2 classes.
Found 3431 images belonging to 2 classes.
Found 1587 images belonging to 2 classes.


AttributeError: 'ModelWrapper' object has no attribute 'model'

In [ ]:
model_wrapper.export_to_png()